## Label Propagation Example (PyG)

In [ ]:
import argparse

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

import torch_geometric.transforms as T

import torch
import torch.nn.functional as F

from utils import EarlyStopping, seed_everything
from model import GNN

import numpy as np

In [ ]:
# import LabelPropagation. Note that LabelPropagation has been implemented in PyG, you can import it by `from torch_geometric.nn import LabelPropagation`
from gtrick.pyg import LabelPropagation

### Define Train Process

In [ ]:
def train(model, data, train_idx, optimizer, task_type):
    model.train()
    y = data.y
    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    if task_type == 'binary classification':
        loss = F.binary_cross_entropy_with_logits(
            out[train_idx], y.squeeze(1)[train_idx])
    elif task_type == 'multiclass classification':
        loss = F.cross_entropy(out[train_idx], y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
@torch.no_grad()
def test(model, data, split_idx, evaluator, eval_metric):
    model.eval()

    y = data.y
    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_metric = evaluator.eval({
        'y_true': y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })[eval_metric]
    valid_metric = evaluator.eval({
        'y_true': y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })[eval_metric]
    test_metric = evaluator.eval({
        'y_true': y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })[eval_metric]

    return train_metric, valid_metric, test_metric, torch.softmax(out, dim=-1)

In [ ]:
def run_node_pred(args, model, dataset):
    device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)

    model.to(device)

    # dataset = DglNodePropPredDataset(name=args.dataset, root=args.dataset_path)
    evaluator = Evaluator(name=args.dataset)

    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    data = data.to(device)

    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train']

    final_test_acc, final_test_acc_lp = [], []
    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        early_stopping = EarlyStopping(
            patience=args.patience, verbose=True, mode='max')

        best_test_acc, best_val_acc = 0, 0
        best_out = None

        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx,
                         optimizer, dataset.task_type)
            result = test(model, data, split_idx,
                          evaluator, dataset.eval_metric)

            train_acc, valid_acc, test_acc, out = result

            if epoch % args.log_steps == 0:
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

            if valid_acc > best_val_acc:
                best_val_acc = valid_acc
                best_test_acc = test_acc
                best_out = out

            if early_stopping(valid_acc, model):
                break
        
        # use LabelPropagation to predict test labels and ensemble results
        lp = LabelPropagation(args.lp_layers, args.lp_alpah)
        yh = lp(data.y, data.adj_t, mask=train_idx)

        y_pred = torch.argmax(best_out + yh, dim=-1, keepdim=True)

        test_acc_lp = evaluator.eval({
            'y_true': data.y[split_idx['test']],
            'y_pred': y_pred[split_idx['test']],
        })[dataset.eval_metric]

        print('Best Test Acc: {:.4f}, Best Test Acc with LP: {:.4f}'.format(best_test_acc, test_acc_lp))
        
        final_test_acc.append(best_test_acc)
        final_test_acc_lp.append(test_acc_lp)
    
    print('Test Acc: {:.4f}  ± {:.4f}, Test Acc with LP: {:.4f}  ± {:.4f}'.format(np.mean(final_test_acc), np.std(final_test_acc), np.mean(final_test_acc_lp), np.std(final_test_acc_lp)))

### Run Experiment

In [ ]:
parser = argparse.ArgumentParser(
    description='train node property prediction')
parser.add_argument('--dataset', type=str, default='ogbn-arxiv',
                    choices=['ogbn-arxiv'])
parser.add_argument('--dataset_path', type=str, default='/dev/dataset',
                    help='path to dataset')
parser.add_argument('--device', type=int, default=1)
parser.add_argument('--log_steps', type=int, default=1)
parser.add_argument('--model', type=str, default='sage')
parser.add_argument('--num_layers', type=int, default=3)
parser.add_argument('--hidden_channels', type=int, default=256)
parser.add_argument('--dropout', type=float, default=0.5)
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epochs', type=int, default=500)
parser.add_argument('--runs', type=int, default=3)
parser.add_argument('--patience', type=int, default=30)

# params for Label Propagation
parser.add_argument('--lp_layers', type=int, default=50)
parser.add_argument('--lp_alpha', type=float, default=0.9)

args = parser.parse_args(args=[])
print(args)

seed_everything(3042)

In [ ]:
dataset = PygNodePropPredDataset(name=args.dataset, transform=T.ToSparseTensor(), root=args.dataset_path)
data = dataset[0]

num_features = data.x.shape[1]

model = GNN(num_features, args.hidden_channels,
                dataset.num_classes, args.num_layers,
                args.dropout, args.model)

In [ ]:
run_node_pred(args, model, dataset)